Establecemos una ``semilla`` que nos permitirá hacer que los experimentos sean reproducibles.

In [89]:
#semilla
seed = 42

Realizamos todas las `importaciones` necesarias.

In [ ]:
#Importamos las librerias
from sklearn.model_selection import train_test_split
from sklearn import set_config
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.model_selection import LeaveOneOut
import numpy as np
import os
from PIL import Image

# 1.Carga de datos

In [91]:
image_dir = 'sin_procesar'

images = []

for filename in os.listdir(image_dir):
    if filename.endswith('.jpg'): 
        img_path = os.path.join(image_dir, filename)
        img = Image.open(img_path)
        images.append(img)

# Print the number of images loaded
print(f'Loaded {len(images)} images.')
#indica el nombre de la imagen en la posicion 2
print(images[2].filename)

Loaded 70 images.
sin_procesar\11.jpg


In [92]:
set_config(transform_output="pandas")
df = pd.read_csv('Hito2Datos.csv', delimiter=';')

In [93]:
# Definir las etiquetas
etiquetas = ['URL imagen', 'ID', 'Inundado', 'transitable', 'formarURL', 'InundadoJm', 'TransitableJm', 'InundadoA', 'TransitableA', 'InundadoI', 'TransitableI', 'InundadoJ', 'TransitableJ', 'InundadoC', 'TransitableC']

# Rellenar las columnas 'Inundado' y 'transitable' con valores de otras columnas
df['Inundado'] = df[['InundadoJm', 'InundadoA', 'InundadoI', 'InundadoJ', 'InundadoC']].mode(axis=1)[0]
df['transitable'] = df[['TransitableJm', 'TransitableA', 'TransitableI', 'TransitableJ', 'TransitableC']].mode(axis=1)[0]

# Mostrar el dataframe actualizado
print(df.head())

  URL imagen  ID Inundado transitable  \
0     Imagen   1       Si          No   
1     Imagen   2       Si          No   
2     Imagen   3       Si          No   
3     Imagen   4       Si          No   
4     Imagen   5       Si          No   

                                           formarURL InundadoJm TransitableJm  \
0  https://raw.githubusercontent.com/alenavarroxp...         Si            No   
1  https://raw.githubusercontent.com/alenavarroxp...         Si            No   
2  https://raw.githubusercontent.com/alenavarroxp...         No            Si   
3  https://raw.githubusercontent.com/alenavarroxp...         Si            No   
4  https://raw.githubusercontent.com/alenavarroxp...         Si            No   

  InundadoA TransitableA InundadoI TransitableI InundadoJ TransitableJ  \
0        Si           No        Si           No        Si           No   
1        Si           No        Si           No        Si           No   
2        Si           No        Si          

Comprobando que se ha cargado correctamente:

In [94]:
df.sample(5, random_state=seed)

,URL imagen,ID,Inundado,transitable,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC
22,Imagen,23,Si,No,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,Si,Si,No,Si,No,Si,No
0,Imagen,1,Si,No,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,No,Si,No,Si,No,Si,No
49,Imagen,50,No,No,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,No,No,No,No,No
4,Imagen,5,Si,No,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,Si,Si,No,Si,No,Si,No
54,Imagen,55,No,Si,https://raw.githubusercontent.com/alenavarroxp...,No,Si,No,Si,No,Si,No,Si,No,Si


A continuación, creamos ``etiquetas`` para separar la variable objetivo del resto de variables.

In [95]:
target = ["transitable", "Inundado"]
columns = target
axis = "columns"

arguments = {"columns": columns, "axis": axis}
X, y = df.drop(**arguments), df[target]

Vamos a comprobar que se ha separado correctamente. Comenzamos con las `variables predictoras`:

In [96]:
X.sample(5, random_state=seed)

,URL imagen,ID,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC
22,Imagen,23,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,Si,Si,No,Si,No,Si,No
0,Imagen,1,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,No,Si,No,Si,No,Si,No
49,Imagen,50,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,No,No,No,No,No
4,Imagen,5,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,Si,Si,No,Si,No,Si,No
54,Imagen,55,https://raw.githubusercontent.com/alenavarroxp...,No,Si,No,Si,No,Si,No,Si,No,Si


Y continuamos con la `variable clase`:

In [97]:
y.sample(5, random_state=seed)

,transitable,Inundado
22,No,Si
0,No,Si
49,No,No
4,No,Si
54,Si,No


Por último, `dividimos` el conjunto de datos en entrenamiento y prueba mediante un *holdout* estratificado:

In [98]:
test_size = 0.10

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed, stratify=y)

In [99]:
# Obtener los nombres de las imágenes en X_train
image_id = X_train['ID'].tolist()
# Imprimir los nombres de las imágenes
print(image_id)
image_names = []
# recorremos la lista de nombres de las images y creamos una lista con las imagenes de image_names
for i in image_id:
    image_names.append(str(i)+'.jpg')
    
images_train = []
for img in images:
    if img.filename.split("\\")[-1] in image_names:
        images_train.append(img)
images_train[0].filename

[70, 54, 40, 4, 63, 36, 31, 18, 7, 68, 55, 69, 64, 48, 14, 52, 8, 20, 23, 11, 6, 39, 22, 16, 46, 53, 38, 28, 44, 17, 59, 32, 66, 29, 49, 42, 35, 30, 51, 3, 57, 26, 27, 21, 19, 62, 43, 13, 60, 5, 67, 33, 34, 65, 24, 37, 25, 47, 15, 61, 10, 41, 9]


'sin_procesar\\10.jpg'

Y nos aseguramos que se ha realizado adecuadamente. Comenzamos con el conjunto de datos de entrenamiento:

In [100]:
X_train.sample(5, random_state=seed)

,URL imagen,ID,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC
40,Imagen,41,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,No,No,No,No,No
46,Imagen,47,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,Si,No,No,No,No
69,Imagen,70,https://raw.githubusercontent.com/alenavarroxp...,No,Si,No,Si,No,Si,No,Si,No,NaN
20,Imagen,21,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,No,Si,No,Si,No,Si,No
35,Imagen,36,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,Si,No,Si,No,No


In [101]:
y_train.sample(5, random_state=seed)

,transitable,Inundado
40,No,No
46,No,No
69,Si,No
20,No,Si
35,No,No


Y finalizamos con el conjunto de datos de prueba:

In [102]:
X_test.sample(5, random_state=seed)

,URL imagen,ID,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC
11,Imagen,12,https://raw.githubusercontent.com/alenavarroxp...,Si,Si,Si,Si,Si,Si,Si,No,Si,No
49,Imagen,50,https://raw.githubusercontent.com/alenavarroxp...,No,No,No,No,No,No,No,No,No,No
57,Imagen,58,https://raw.githubusercontent.com/alenavarroxp...,No,Si,No,Si,No,Si,No,Si,No,Si
1,Imagen,2,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,No,Si,No,Si,No,Si,No
0,Imagen,1,https://raw.githubusercontent.com/alenavarroxp...,Si,No,Si,No,Si,No,Si,No,Si,No


In [103]:
y_test.sample(5, random_state=seed)

,transitable,Inundado
11,Si,Si
49,No,No
57,Si,No
1,No,Si
0,No,Si


Se borran las etiquetas que no aportan informacion

In [104]:
etiquetas=['URL imagen,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC'] 

In [105]:
print(etiquetas)

['URL imagen,formarURL,InundadoJm,TransitableJm,InundadoA,TransitableA,InundadoI,TransitableI,InundadoJ,TransitableJ,InundadoC,TransitableC']


In [106]:
print(X_train.columns)

Index(['URL imagen', 'ID', 'formarURL', 'InundadoJm', 'TransitableJm',
       'InundadoA', 'TransitableA', 'InundadoI', 'TransitableI', 'InundadoJ',
       'TransitableJ', 'InundadoC', 'TransitableC'],
      dtype='object')


# ELIMINACIÓN DE VARIABLES

vamos a `eliminar` las variables que no aportan información con la ayuda de un `pipeline`.

In [107]:
def drop_columns(X, etiquetas):
    # Convertir el string de 'etiquetas' en una lista de columnas
    etiquetas = etiquetas[0].split(',')
    
    # Eliminar las columnas que existen en el dataframe
    return X.drop(columns=etiquetas)

In [108]:
X_train=drop_columns(X_train, etiquetas)
X_train.sample(5, random_state=seed)

,ID
40,41
46,47
69,70
20,21
35,36


In [109]:
X_test=drop_columns(X_test, etiquetas)
X_test.sample(5, random_state=seed)

,ID
11,12
49,50
57,58
1,2
0,1


# ENCODER y_train e y_test

In [110]:
y_train = y_train.applymap(lambda x: 1 if x == "Si" else (0 if x == "No" else x))
y_train.head()

C:\Users\Juanmi\AppData\Local\Temp\ipykernel_25324\1745860845.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  y_train = y_train.applymap(lambda x: 1 if x == "Si" else (0 if x == "No" else x))


,transitable,Inundado
69,1,0
53,1,0
39,0,0
3,0,1
62,1,0


In [111]:
y_test = y_test.applymap(lambda x: 1 if x == "Si" else (0 if x == "No" else x))
y_test.head()

C:\Users\Juanmi\AppData\Local\Temp\ipykernel_25324\3414226292.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  y_test = y_test.applymap(lambda x: 1 if x == "Si" else (0 if x == "No" else x))


,transitable,Inundado
11,1,1
49,0,0
1,0,1
44,0,0
0,0,1


# TRANSFORMACION DE LAS IMÁGENES

In [112]:
# Convertir imágenes a escala de grises y rotarlas
augmented_images = []

for img in images_train:
    # Convertir a escala de grises
    gray_img = img.convert('L')
    gray_img.filename = "0_"+img.filename.split("\\")[-1]
    
    # Rotar la imagen 90º, 180º y 270º
    rotated_90 = gray_img.rotate(90)
    rotated_180 = gray_img.rotate(180)
    rotated_270 = gray_img.rotate(270)
    rotated_90.filename = "90_"+img.filename.split("\\")[-1]
    rotated_180.filename = "180_"+img.filename.split("\\")[-1]
    rotated_270.filename = "270_"+img.filename.split("\\")[-1]
    
    # Añadir las imágenes originales y transformadas a la lista con filenames
    augmented_images.extend([gray_img, rotated_90, rotated_180, rotated_270])

# Imprimir el número de imágenes aumentadas
print(f'Número total de imágenes aumentadas: {len(augmented_images)}')
augmented_images[2].filename

Número total de imágenes aumentadas: 252


'180_10.jpg'

In [113]:
output_dir = 'trainprocesados'
os.makedirs(output_dir, exist_ok=True)

for img in augmented_images:
    img.save(os.path.join(output_dir, img.filename.split("\\")[-1]))

print(f'Se han guardado {len(augmented_images)} imágenes en la carpeta {output_dir}.')

Se han guardado 252 imágenes en la carpeta trainprocesados.


In [114]:
# Crear la carpeta MetaData si no existe
os.makedirs('MetaData', exist_ok=True)

# Juntar X_train e y_train
train_data = pd.concat([X_train, y_train], axis=1)


# Juntar X_test e y_test
test_data = pd.concat([X_test, y_test], axis=1)
test_data.to_csv('MetaData/test_data.csv', index=False)


# test_data se modifica para tener referencia a las imagenes de la carpeta de trainprocesados

In [115]:
angles = [0, 90, 180, 270]
expanded_data = []
for _, row in train_data.iterrows():
    for angle in angles:
        expanded_data.append({
            "ID": f"{angle}_{row['ID']}",
            "transitable": row["transitable"],
            "Inundado": row["Inundado"]
        })

# Convertir a DataFrame
expanded_train_data = pd.DataFrame(expanded_data)

In [116]:
expanded_train_data.to_csv('MetaData/train_data.csv', index=False)


print('Los archivos CSV se han guardado en la carpeta MetaData.')

Los archivos CSV se han guardado en la carpeta MetaData.


# Para generar la validacion se utilizara Leave-One-Out (LOO), por lo que no se entregaran los datos de train,validadicon dado que para cada iteracion se utilizara un solo dato para validar y el resto para entrenar.

In [ ]:
# Crear el objeto LeaveOneOut
loo = LeaveOneOut()

# Separar las características y las etiquetas
X = expanded_train_data.drop(columns=['transitable', 'Inundado'])
y = expanded_train_data[['transitable', 'Inundado']]

# Convertir a numpy arrays
X = X.values
y = y.values

# Iterar sobre cada split de LOO
#for train_index, test_index in loo.split(X):
#    X_train, X_test = X[train_index], X[test_index]
#    y_train, y_test = y[train_index], y[test_index]
    
    # aqui se continuaria con el entrenamiento del modelo